In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q sentence-transformers==2.7.0 transformers==4.45.0

In [ ]:
import os
import torch
import random

seed=

os.environ['PYTHONHASHSEED']=str(seed)



In [ ]:
import os, re, ast, gc, warnings
from typing import Tuple, List
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor, Pool
from sklearn.neighbors import BallTree
from scipy import sparse
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
RANDOM_STATE = 42
N_FOLDS = 5
USE_GPU = False

TFIDF_MAX_FEATURES = 120_000
MIN_DF = 3
MAX_DF = 0.975
NGRAM_RANGE = (1, 2)
SVD_COMPONENTS = 200
USE_CHAR_TFIDF = True 

USE_KEYWORD_FLAGS = True
KEYWORD_MAX_DF = 0.85
KEYWORD_TOP_K  = 350
TOKEN_PATTERN  = r"(?u)\b[а-яa-z0-9][а-яa-z0-9\-]{2,}\b"

USE_RUBERT = True
BERT_MODEL = "cointegrated/rubert-tiny2"   # можно sbert_large_nlu_ru если успею
BERT_BATCH_SIZE = 256
BERT_NORMALIZE = True
BERT_SVD_COMPONENTS = 96
BERT_RAW_KEEP_DIMS = 64 
RIDGE_ALPHA  = 3.0
RIDGE_SOLVER = "lsqr"

In [ ]:
MOSCOW_CENTER_LAT, MOSCOW_CENTER_LON = 55.752023, 37.617499 

In [ ]:
TRAIN_PATH = "/kaggle/input/vseros-geo-task/train.tsv"
TEST_PATH  = "/kaggle/input/vseros-geo-task/test.tsv"
REVIEWS_PATH_CANDIDATES = ["/kaggle/input/vseros-geo-task/reviews.txv/reviews.tsv"]
SUBMISSION_PATH = "submission_catboost_tfidf.csv"

In [ ]:
def smart_read_table(path, sep="\t", **kwargs):
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    try:
        return pd.read_csv(path, sep=sep, **kwargs)
    

def try_read_reviews(candidates: List[str]) -> pd.DataFrame:
    for p in candidates:
        if os.path.exists(p):
            for sep in ["\t", ","]:
                try:
                    df = pd.read_csv(p, sep=sep)
                    if {"id","text"}.issubset(df.columns):
                        return df[["id","text"]]

def parse_coordinates_to_lon_lat(s) -> Tuple[float, float]:
    if pd.isna(s):
        return np.nan, np.nan
    try:
        val = ast.literal_eval(s)
        if isinstance(val, (list, tuple)) and len(val) == 2:
            return float(val[0]), float(val[1])
    except Exception:
        pass
    m = re.findall(r"[-+]?\d*\.\d+|[-+]?\d+", str(s))
    if len(m) >= 2:
        return float(m[0]), float(m[1])
    return np.nan, np.nan

def add_share_pair(df: pd.DataFrame, a: str, b: str, new_col: str):
    if a in df.columns and b in df.columns:
        denom = (pd.to_numeric(df[a], errors="coerce").fillna(0) +
                 pd.to_numeric(df[b], errors="coerce").fillna(0))
        num = pd.to_numeric(df[a], errors="coerce").fillna(0)
        df[new_col] = np.where(denom > 0, num / denom, np.nan)

def light_log1p_skewed(df: pd.DataFrame, numeric_cols: List[str], skew_thr: float = 1.0) -> List[str]:
    transformed = []
    for c in numeric_cols:
        try:
            s = pd.to_numeric(df[c], errors="coerce")
            if s.notna().sum() == 0: continue
            if (s >= 0).all():
                skew = s.skew(skipna=True)
                if abs(skew) > skew_thr:
                    df[c] = np.log1p(s)
                    transformed.append(c)
        except Exception:
            continue
    return transformed

R_EARTH = 6_371_000.0
def _to_rad_series(df):
    lat = pd.to_numeric(df["lat"], errors="coerce").values
    lon = pd.to_numeric(df["lon"], errors="coerce").values
    lat_rad = np.deg2rad(lat); lon_rad = np.deg2rad(lon)
    valid = np.isfinite(lat_rad) & np.isfinite(lon_rad)
    return lat_rad, lon_rad, valid

def _safe_div(a, b):
    b = np.asarray(b)
    return np.where(b != 0, np.asarray(a) / b, np.nan)

def haversine_dist_m(lat1, lon1, lat2, lon2):
    lat1 = np.deg2rad(lat1); lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2); lon2 = np.deg2rad(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*(np.sin(dlon/2.0)**2)
    c = 2*np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R_EARTH * c

In [ ]:
train = smart_read_table(TRAIN_PATH, sep="\t")
test  = smart_read_table(TEST_PATH,  sep="\t")
assert {"id","target"}.issubset(train.columns) and "id" in test.columns

In [ ]:
KEYWORD_MIN_DF = max(8, int(0.004 * len(train)))

train = train[train["target"].fillna(0) > 0].reset_index(drop=True)

In [ ]:
for df in (train, test):
    if "coordinates" in df.columns:
        lon_lat = df["coordinates"].apply(parse_coordinates_to_lon_lat)
        df["lon"] = [x[0] for x in lon_lat]
        df["lat"] = [x[1] for x in lon_lat]

print("Чтение и агрегация отзывов...")
reviews = try_read_reviews(REVIEWS_PATH_CANDIDATES)
reviews["text"] = reviews["text"].astype(str).fillna("")
agg = (
    reviews.groupby("id")["text"]
    .agg([
        ("reviews_text", lambda s: " ".join(s.astype(str))),
        ("reviews_count", "size"),
        ("reviews_avg_len", lambda s: np.mean([len(t) for t in s.astype(str)]))
    ])
    .reset_index()
)
train = train.merge(agg, on="id", how="left")
test  = test.merge(agg, on="id",  how="left")
for df in (train, test):
    df["reviews_text"] = df["reviews_text"].fillna("")
    df["reviews_count"] = df["reviews_count"].fillna(0).astype(int)
    df["reviews_avg_len"] = df["reviews_avg_len"].fillna(0.0)

In [ ]:
if USE_KEYWORD_FLAGS:
    vect = CountVectorizer(
        binary=True, min_df=KEYWORD_MIN_DF, max_df=KEYWORD_MAX_DF,
        lowercase=True, token_pattern=TOKEN_PATTERN
    )
    X_tr_bin = vect.fit_transform(train["reviews_text"].astype(str))
    X_te_bin = vect.transform(test["reviews_text"].astype(str))
    feats = vect.get_feature_names_out()

    y_arr = train["target"].astype(float).values
    n = len(y_arr); y_mean = y_arr.mean(); vary = y_arr.var() + 1e-12
    p = (X_tr_bin.sum(axis=0).A1) / n
    xy = (X_tr_bin.T @ y_arr) / n
    cov = xy - y_mean * p
    varx = p * (1 - p) + 1e-12
    corr = cov / np.sqrt(varx * vary)

    top_idx = np.argsort(-np.abs(corr))[:KEYWORD_TOP_K]
    sel_feats = feats[top_idx]
    X_tr_sel = X_tr_bin[:, top_idx]
    X_te_sel = X_te_bin[:, top_idx]

    sel_cols = [f"kw_{i}_{t}" for i, t in enumerate(sel_feats)]
    train_kw = pd.DataFrame(X_tr_sel.toarray().astype(np.int8), index=train.index, columns=sel_cols)
    test_kw  = pd.DataFrame(X_te_sel.toarray().astype(np.int8), index=test.index,  columns=sel_cols)

    MANUAL_REGEX = {
        "kw_vkus": r"\bвкус\w*", "kw_chist": r"\bчист\w*", "kw_svezh": r"\bсвеж\w*",
        "kw_bystr": r"\bбыстр\w*", "kw_medlen": r"\bмедлен\w*", "kw_dorog": r"\bдорог\w*",
        "kw_deshev": r"\bдешев\w*", "kw_holodn": r"\bхолодн\w*", "kw_goryach": r"\bгоряч\w*",
        "kw_ochered": r"\bочеред\w*", "kw_vejl": r"\bвежлив\w*", "kw_gryb": r"\bгруб\w*",
        "kw_rekom": r"\bрекоменд\w*", "kw_ne_rekom": r"не\s*рекоменд\w*", "kw_servis": r"\bсервис\w*",
        "kw_uzhast": r"\bужас\w*", "kw_otlich": r"\bотлич\w*",
    }
    for col, pattern in MANUAL_REGEX.items():
        train_kw[col] = train["reviews_text"].str.contains(pattern, case=False, regex=True).fillna(False).astype(np.int8)
        test_kw[col]  = test["reviews_text"].str.contains(pattern, case=False, regex=True).fillna(False).astype(np.int8)

    POS_KEYS = ["kw_vkus","kw_chist","kw_svezh","kw_bystr","kw_vejl","kw_rekom","kw_otlich","kw_servis"]
    NEG_KEYS = ["kw_gryb","kw_medlen","kw_dorog","kw_holodn","kw_ochered","kw_uzhast","kw_ne_rekom"]
    for df_kw in (train_kw, test_kw):
        pos_cols = [c for c in POS_KEYS if c in df_kw.columns]
        neg_cols = [c for c in NEG_KEYS if c in df_kw.columns]
        df_kw["kw_pos_hits"] = df_kw[pos_cols].sum(axis=1).astype(np.int16) if pos_cols else 0
        df_kw["kw_neg_hits"] = df_kw[neg_cols].sum(axis=1).astype(np.int16) if neg_cols else 0
        df_kw["kw_pos_any"] = (df_kw["kw_pos_hits"] > 0).astype(np.int8)
        df_kw["kw_neg_any"] = (df_kw["kw_neg_hits"] > 0).astype(np.int8)

    train = pd.concat([train, train_kw], axis=1)
    test  = pd.concat([test,  test_kw],  axis=1)
    del X_tr_bin, X_te_bin, X_tr_sel, X_te_sel, train_kw, test_kw; gc.collect()

In [ ]:
print("Строим TF-IDF (word + optional char) + SVD...")
corpus = pd.concat([train["reviews_text"], test["reviews_text"]], axis=0).astype(str)

# Word TF-IDF
tfidf_word = TfidfVectorizer(
    max_features=TFIDF_MAX_FEATURES,
    min_df=MIN_DF,
    max_df=MAX_DF,
    ngram_range=NGRAM_RANGE,
    lowercase=True,
    strip_accents="unicode",
    sublinear_tf=True
)
Xw = tfidf_word.fit_transform(corpus.values)

if USE_CHAR_TFIDF:
    tfidf_char = TfidfVectorizer(
        analyzer="char",
        ngram_range=(3,5),
        min_df=3,
        max_df=0.98,
        lowercase=True,
        sublinear_tf=True
    )
    Xc = tfidf_char.fit_transform(corpus.values)
    X_all = sparse.hstack([Xw, Xc]).tocsr()
    del Xw, Xc
else:
    X_all = Xw

svd = TruncatedSVD(n_components=SVD_COMPONENTS, random_state=RANDOM_STATE)
X_svd = svd.fit_transform(X_all)
del X_all; gc.collect()

X_svd_train = X_svd[:len(train)]
X_svd_test  = X_svd[len(train):]
svd_cols = [f"text_svd_{i}" for i in range(X_svd.shape[1])]
for i, col in enumerate(svd_cols):
    train[col] = X_svd_train[:, i]
    test[col]  = X_svd_test[:, i]
del X_svd, X_svd_train, X_svd_test, corpus; gc.collect()

In [ ]:
pairs = [
    ("female_300m","male_300m","female_share_300m"),
    ("female_1000m","male_1000m","female_share_1000m"),
    ("employed_300m","unemployed_300m","employed_share_300m"),
    ("employed_1000m","unemployed_1000m","employed_share_1000m"),
    ("married_300m","not_married_300m","married_share_300m"),
    ("married_1000m","not_married_1000m","married_share_1000m"),
    ("has_children_300m","no_children_300m","has_children_share_300m"),
    ("has_children_1000m","no_children_1000m","has_children_share_1000m"),
    ("higher_education_300m","no_higher_education_300m","higher_education_share_300m"),
    ("higher_education_1000m","no_higher_education_1000m","higher_education_share_1000m"),
]
for (a,b,nc) in pairs:
    add_share_pair(train, a, b, nc)
    add_share_pair(test,  a, b, nc)

In [ ]:
def samecat_counts_multi_r(train_df, test_df, radii_m=(100,200,300)):
    all_df = pd.concat([
        train_df[["id","lat","lon","category"]],
        test_df[["id","lat","lon","category"]]
    ], axis=0, ignore_index=True)
    all_df["category"] = all_df["category"].astype(str).fillna("Unknown")
    lat_rad, lon_rad, valid = _to_rad_series(all_df)
    X_valid = np.c_[lat_rad[valid], lon_rad[valid]]
    idx_valid = np.where(valid)[0]; cats = all_df["category"].values
    tree = BallTree(X_valid, metric="haversine")
    out = {r: np.full(len(all_df), np.nan, dtype=np.float32) for r in radii_m}
    for r_m in radii_m:
        neigh = tree.query_radius(X_valid, r=r_m/R_EARTH, return_distance=False)
        cnt = np.zeros(len(idx_valid), dtype=np.int32)
        for j, nb_local in enumerate(neigh):
            gi = idx_valid[j]
            if nb_local.size == 0: cnt[j] = 0; continue
            nb_global = idx_valid[nb_local]; nb_global = nb_global[nb_global != gi]
            if nb_global.size == 0: cnt[j] = 0; continue
            cnt[j] = int(np.sum(cats[nb_global] == cats[gi]))
        res = np.full(len(all_df), np.nan, dtype=np.float32); res[idx_valid] = cnt
        out[r_m] = res
    ntr = len(train_df)
    res_train = {r: out[r][:ntr] for r in radii_m}; res_test  = {r: out[r][ntr:] for r in radii_m}
    return res_train, res_test

cnt_tr, cnt_te = samecat_counts_multi_r(train, test, radii_m=(100,200,300))
for r_m in (100,200,300):
    train[f"samecat_cnt_r{r_m}"] = cnt_tr[r_m]; test[f"samecat_cnt_r{r_m}"]  = cnt_te[r_m]

In [ ]:
def oof_local_target_stats_by_cat(train_df, test_df, radii_m=(200,300), n_splits=N_FOLDS, seed=RANDOM_STATE):
    y = train_df["target"].astype(float).values
    cat_tr = train_df["category"].astype(str).fillna("Unknown").values
    cat_te = test_df["category"].astype(str).fillna("Unknown").values
    lat_tr = pd.to_numeric(train_df["lat"], errors="coerce").values
    lon_tr = pd.to_numeric(train_df["lon"], errors="coerce").values
    lat_te = pd.to_numeric(test_df["lat"],  errors="coerce").values
    lon_te = pd.to_numeric(test_df["lon"],  errors="coerce").values

    bins = pd.qcut(y, q=min(10, max(2, len(y)//50)), duplicates="drop")
    y_bins = pd.factorize(bins)[0]
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    res_tr = {r: dict(mean=np.full(len(train_df), np.nan, dtype=np.float32),
                      median=np.full(len(train_df), np.nan, dtype=np.float32),
                      count=np.zeros(len(train_df), dtype=np.float32)) for r in radii_m}
    res_te = {r: dict(mean=np.full(len(test_df), np.nan, dtype=np.float32),
                      median=np.full(len(test_df), np.nan, dtype=np.float32),
                      count=np.zeros(len(test_df), dtype=np.float32)) for r in radii_m}

    valid_tr = np.isfinite(lat_tr) & np.isfinite(lon_tr)
    valid_te = np.isfinite(lat_te) & np.isfinite(lon_te)
    X_tr_all = np.c_[np.deg2rad(np.where(valid_tr, lat_tr, 0.0)), np.deg2rad(np.where(valid_tr, lon_tr, 0.0))]
    X_te_all = np.c_[np.deg2rad(np.where(valid_te, lat_te, 0.0)), np.deg2rad(np.where(valid_te, lon_te, 0.0))]

    for r_m in radii_m:
        r = r_m / R_EARTH
        for tr_idx, va_idx in skf.split(train_df, y_bins):
            tr_idx_valid = tr_idx[valid_tr[tr_idx]]
            va_idx_valid = va_idx[valid_tr[va_idx]]
            if len(tr_idx_valid) == 0 or len(va_idx_valid) == 0: continue
            Xtr = X_tr_all[tr_idx_valid]; ytr = y[tr_idx_valid]
            cat_tr_fold = cat_tr[tr_idx_valid]; cat_va_fold = cat_tr[va_idx_valid]
            tree = BallTree(Xtr, metric="haversine")
            neigh = tree.query_radius(X_tr_all[va_idx_valid], r=r, return_distance=False)
            for j, nb in enumerate(neigh):
                if nb.size == 0: continue
                nb = nb[cat_tr_fold[nb] == cat_va_fold[j]]
                if nb.size == 0: continue
                vals = ytr[nb]
                res_tr[r_m]["mean"][va_idx_valid[j]] = float(np.mean(vals))
                res_tr[r_m]["median"][va_idx_valid[j]] = float(np.median(vals))
                res_tr[r_m]["count"][va_idx_valid[j]]  = float(len(vals))
        tr_all_valid_idx = np.where(valid_tr)[0]
        if len(tr_all_valid_idx) > 0 and np.any(valid_te):
            Xtr = X_tr_all[tr_all_valid_idx]; ytr = y[tr_all_valid_idx]
            cat_tr_all = cat_tr[tr_all_valid_idx]
            tree_full = BallTree(Xtr, metric="haversine")
            te_valid_idx = np.where(valid_te)[0]
            neigh_te = tree_full.query_radius(X_te_all[te_valid_idx], r=r, return_distance=False)
            for j, nb in enumerate(neigh_te):
                if nb.size == 0: continue
                idx = te_valid_idx[j]
                nb = nb[cat_tr_all[nb] == cat_te[idx]]
                if nb.size == 0: continue
                vals = ytr[nb]
                res_te[r_m]["mean"][idx]   = float(np.mean(vals))
                res_te[r_m]["median"][idx] = float(np.median(vals))
                res_te[r_m]["count"][idx]  = float(len(vals))
    return res_tr, res_te

loc_tr, loc_te = oof_local_target_stats_by_cat(train, test, radii_m=(200,300), n_splits=N_FOLDS, seed=RANDOM_STATE)

In [ ]:
for r_m in (200, 300):
    train[f"loc_cat_mean_r{r_m}"]   = loc_tr[r_m]["mean"]
    train[f"loc_cat_median_r{r_m}"] = loc_tr[r_m]["median"]
    train[f"loc_cat_count_r{r_m}"]  = loc_tr[r_m]["count"]
    test[f"loc_cat_mean_r{r_m}"]    = loc_te[r_m]["mean"]
    test[f"loc_cat_median_r{r_m}"]  = loc_te[r_m]["median"]
    test[f"loc_cat_count_r{r_m}"]   = loc_te[r_m]["count"]

In [ ]:
def add_location_interactions(df: pd.DataFrame):
    for suf in ("300m","1000m"):
        if f"traffic_{suf}" in df.columns and f"homes_{suf}" in df.columns:
            df[f"traffic_per_home_{suf}"] = _safe_div(pd.to_numeric(df[f"traffic_{suf}"], errors="coerce"),
                                                      (pd.to_numeric(df[f"homes_{suf}"], errors="coerce") + 1))
        if f"works_{suf}" in df.columns and f"homes_{suf}" in df.columns:
            df[f"works_to_homes_{suf}"] = _safe_div(pd.to_numeric(df[f"works_{suf}"], errors="coerce"),
                                                    (pd.to_numeric(df[f"homes_{suf}"], errors="coerce") + 1))
    income_cols = ["below_average_income","average_income","above_average_income","high_income","premium_income"]
    for rad in ("300m","1000m"):
        present = [c for c in income_cols if f"{c}_{rad}" in df.columns]
        if present:
            total = np.zeros(len(df), dtype=np.float64)
            for c in present:
                total += pd.to_numeric(df[f"{c}_{rad}"], errors="coerce").fillna(0).values
            total[total == 0] = np.nan
            if f"below_average_income_{rad}" in df.columns:
                df[f"low_income_share_{rad}"] = pd.to_numeric(df[f"below_average_income_{rad}"], errors="coerce") / total
            hi_sum = np.zeros(len(df), dtype=np.float64)
            for c in ("high_income","premium_income"):
                if f"{c}_{rad}" in df.columns:
                    hi_sum += pd.to_numeric(df[f"{c}_{rad}"], errors="coerce").fillna(0).values
            df[f"high_income_share_{rad}"] = _safe_div(hi_sum, total)
    for r_m in (200, 300):
        if f"samecat_cnt_r{r_m}" in df.columns:
            if "low_income_share_300m" in df.columns:
                df[f"samecat_x_lowincome_r{r_m}"]  = df[f"samecat_cnt_r{r_m}"] * df["low_income_share_300m"]
            if "high_income_share_300m" in df.columns:
                df[f"samecat_x_highincome_r{r_m}"] = df[f"samecat_cnt_r{r_m}"] * df["high_income_share_300m"]

add_location_interactions(train); add_location_interactions(test)

In [ ]:
def add_dist_to_center(train_df, test_df, lat_col="lat", lon_col="lon",
                       center_lat=MOSCOW_CENTER_LAT, center_lon=MOSCOW_CENTER_LON):
    for df in (train_df, test_df):
        df["dist_to_center_m"] = haversine_dist_m(
            pd.to_numeric(df[lat_col], errors="coerce"),
            pd.to_numeric(df[lon_col], errors="coerce"),
            center_lat, center_lon
        )

add_dist_to_center(train, test)

In [ ]:
METRO_STATIONS = [
    ("Охотный ряд",55.75703,37.61614), ("Октябрьское поле",55.793615,37.493496),
    ("Октябрьская",55.729,37.61139), ("Орехово",55.61214,37.69584),
    ("Отрадное",55.86417,37.60488), ("Парк культуры",55.73512,37.59328),
    ("Парк Победы (стр.)",55.736559,37.512591), ("Партизанская",55.78962,37.7479),
    ("Павелецкая",55.7313,37.63612), ("Печатники",55.69252,37.7295),
    ("Перово",55.75109,37.78854), ("Первомайская",55.79342,37.79979),
    ("Петровско-Разумовская",55.83712,37.57349), ("Пионерская",55.73583,37.46731),
    ("Планерная",55.85931,37.43687), ("Площадь Ильича",55.745663,37.681123),
    ("Улица Подбельского",55.81503,37.73209), ("Улица Подбельского",55.81336,37.73524),
    ("Полежаевская",55.77691,37.51692), ("Полянка",55.73654,37.61856),
    ("Пражская",55.61354,37.60499), ("Преображенская площадь",55.79655,37.71591),
    ("Профсоюзная",55.67822,37.56381), ("Пролетарская",55.73171,37.66726),
    ("Пронская (стр.)",55.698344,37.850869), ("Пушкинская",55.76565,37.60417),
    ("Речной вокзал",55.85378,37.47679), ("Площадь Революции",55.75646,37.62321),
    ("Рижская",55.79222,37.63557), ("Римская",55.746487,37.682631),
    ("Рязанский проспект",55.71753,37.79425), ("Cавеловская",55.79421,37.58666),
    ("Щелковская",55.80955,37.79884), ("Щукинская",55.80796,37.46629),
    ("Cеменовская",55.78279,37.71844), ("Cерпуховская",55.72658,37.62462),
    ("Cевастопольская",55.65121,37.59939), ("Шаболовская",55.71886,37.60797),
    ("Cходненская",55.84937,37.43951), ("Cлавянский бульвар (стр.)",55.729508,37.468829),
    ("Cмоленская",55.74823,37.58384), ("Cокол",55.80518,37.51495),
    ("Cокольники",55.78893,37.67943), ("Cпортивная",55.72397,37.56547),
    ("Cретенский бульвар (стр.)",55.765551,37.635261), ("Cтрогино (стр.)",55.80435,37.396363),
    ("Cтуденческая",55.73873,37.54825), ("Cухаревская",55.77211,37.63239),
    ("Площадь Cуворова (стр.)",55.781984,37.614487), ("Cвиблово",55.85543,37.65419),
    ("Таганская",55.74255,37.65389), ("Театральная",55.75857,37.6177),
    ("Текстильшики",55.70947,37.73282), ("Теплый стан",55.61814,37.50814),
    ("Тимирязевская",55.81842,37.57571), ("Третьяковская",55.74061,37.62492),
    ("Трубная (стр.)",55.767605,37.6221), ("Царицино",55.62011,37.66939),
    ("Цветной бульвар",55.7716,37.62058), ("Тульская",55.70901,37.6226),
    ("Тургеневская",55.7646,37.63623), ("Тушинская",55.8258,37.43621),
    ("Тверская",55.7652,37.60352), ("Улица 1905 года",55.76355,37.56375),
    ("Университет",55.69167,37.53433), ("Варшавская",55.65381,37.62084),
    ("ВДНХ",55.82177,37.64107), ("Проспект Вернадского",55.67613,37.5045),
    ("Владыкино",55.84669,37.59251), ("Водный стадион",55.8386,37.48749),
    ("Войковская",55.81811,37.49905), ("Волоколамская (стр.)",55.83459,37.38367),
    ("Волгоградский проспект",55.7243,37.68795), ("Волжская",55.69101,37.75498),
    ("Воробьёвы горы",55.710454,37.558601), ("Выхино",55.715,37.81802),
    ("Ясенево",55.60535,37.53494), ("Юго-западная",55.66464,37.48421)
]
metro_df = pd.DataFrame(METRO_STATIONS, columns=["name","lat","lon"])

In [ ]:
def add_nearest_poi_features(train_df, test_df, poi_df,
                             lat_col="lat", lon_col="lon",
                             ks=(1,3,5), radii=(300, 500, 1000),
                             prefix="metro"):
    if poi_df is None or len(poi_df)==0:
        print(f"[{prefix}] empty — skipping.")
        return
    poi_lat = np.deg2rad(pd.to_numeric(poi_df["lat"], errors="coerce").values)
    poi_lon = np.deg2rad(pd.to_numeric(poi_df["lon"], errors="coerce").values)
    mask_poi = np.isfinite(poi_lat) & np.isfinite(poi_lon)
    X_poi = np.c_[poi_lat[mask_poi], poi_lon[mask_poi]]
    if X_poi.shape[0] == 0:
        print(f"[{prefix}] no valid points — skipping.")
        return
    tree = BallTree(X_poi, metric="haversine")

    def _process(df):
        lat = pd.to_numeric(df[lat_col], errors="coerce").values
        lon = pd.to_numeric(df[lon_col], errors="coerce").values
        valid = np.isfinite(lat) & np.isfinite(lon)
        X = np.c_[np.deg2rad(np.where(valid, lat, 0.0)),
                  np.deg2rad(np.where(valid, lon, 0.0))]

        for k in ks:
            k_eff = min(k, X_poi.shape[0])
            dist, _ = tree.query(X[valid], k=k_eff)
            dist_m = dist * R_EARTH
            if k_eff == 1:
                out = np.full(len(df), np.nan, dtype=np.float32)
                out[valid] = dist_m[:, 0]
                df[f"{prefix}_dist_min_m"] = out
            out_mean = np.full(len(df), np.nan, dtype=np.float32)
            out_mean[valid] = dist_m.mean(axis=1)
            df[f"{prefix}_mean_k{k_eff}_m"] = out_mean

        for r in radii:
            ind = tree.query_radius(X[valid], r=(r / R_EARTH), count_only=False, return_distance=False)
            cnt = np.zeros(np.sum(valid), dtype=np.int32)
            for i, nb in enumerate(ind):
                cnt[i] = len(nb)
            out = np.full(len(df), np.nan, dtype=np.float32)
            out[valid] = cnt
            df[f"{prefix}_cnt_r{r}"] = out

    _process(train_df); _process(test_df)

add_nearest_poi_features(train, test, metro_df, ks=(1,3,5), radii=(300,500,1000), prefix="metro")

In [ ]:
for df in (train, test):
    if "traffic_300m" in df.columns:
        for r in (300, 500, 1000):
            df[f"metro_per1k_traffic_r{r}"] = _safe_div(
                df.get(f"metro_cnt_r{r}", np.nan),
                _safe_div(pd.to_numeric(df["traffic_300m"], errors="coerce"), 1000.0)
            )

In [ ]:
if USE_RUBERT:
    try:
        import torch
        from sentence_transformers import SentenceTransformer
        from sklearn.linear_model import Ridge

        train_ids_set = set(train["id"].tolist())
        test_ids_set  = set(test["id"].tolist())

        tr_rev = reviews[reviews["id"].isin(train_ids_set)].merge(
            train[["id","target"]], on="id", how="left"
        ).dropna(subset=["target"]).reset_index(drop=True)
        te_rev = reviews[reviews["id"].isin(test_ids_set)].reset_index(drop=True)

        device = "cuda" if (USE_GPU and torch.cuda.is_available()) else "cpu"
        print(f"[RuBERT] Loading model: {BERT_MODEL} on {device}")
        bert = SentenceTransformer(BERT_MODEL, device=device)

        def encode_texts(texts):
            return bert.encode(
                list(texts),
                batch_size=BERT_BATCH_SIZE,
                convert_to_numpy=True,
                show_progress_bar=True,
                normalize_embeddings=BERT_NORMALIZE,
            ).astype(np.float32)

        print("[RuBERT] Encoding train reviews...")
        X_tr_rev = encode_texts(tr_rev["text"].astype(str))
        print("[RuBERT] Encoding test reviews...")
        X_te_rev = encode_texts(te_rev["text"].astype(str))
        y_rev = tr_rev["target"].astype(float).values

        uniq_ids = train["id"].values
        uniq_targets = train["target"].astype(float).values
        bins_ids = pd.qcut(uniq_targets, q=min(10, max(2, len(uniq_targets)//50)), duplicates="drop")
        y_bins_ids = pd.factorize(bins_ids)[0]
        skf_ids = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

        oof_pred_rev = np.zeros(len(tr_rev), dtype=np.float32)
        te_pred_accum = np.zeros(len(te_rev), dtype=np.float32)
        id_arr_tr = tr_rev["id"].values

        print("[RuBERT] OOF training Ridge head...")
        for fold, (tr_id_idx, va_id_idx) in enumerate(skf_ids.split(uniq_ids, y_bins_ids), 1):
            tr_ids_fold = set(uniq_ids[tr_id_idx]); va_ids_fold = set(uniq_ids[va_id_idx])
            tr_mask = np.isin(id_arr_tr, list(tr_ids_fold))
            va_mask = np.isin(id_arr_tr, list(va_ids_fold))

            head = Ridge(alpha=RIDGE_ALPHA, solver=RIDGE_SOLVER, tol=1e-3, max_iter=2000, fit_intercept=True)
            head.fit(X_tr_rev[tr_mask], y_rev[tr_mask])

            oof_pred_rev[va_mask] = head.predict(X_tr_rev[va_mask]).astype(np.float32)
            te_pred_accum += head.predict(X_te_rev).astype(np.float32) / N_FOLDS

            print(f"[RuBERT][Fold {fold}] train_reviews={tr_mask.sum()}, val_reviews={va_mask.sum()}")

        tr_rev["rev_bert_oof"] = oof_pred_rev
        te_rev["rev_bert_pred"] = te_pred_accum

        tr_agg = (
            tr_rev.groupby("id")["rev_bert_oof"]
            .agg(["mean","median","std","min","max","size"])
            .reset_index()
            .rename(columns={
                "mean":"rev_bert_mean","median":"rev_bert_median","std":"rev_bert_std",
                "min":"rev_bert_min","max":"rev_bert_max","size":"rev_bert_cnt"
            })
        )
        te_agg = (
            te_rev.groupby("id")["rev_bert_pred"]
            .agg(["mean","median","std","min","max","size"])
            .reset_index()
            .rename(columns={
                "mean":"rev_bert_mean","median":"rev_bert_median","std":"rev_bert_std",
                "min":"rev_bert_min","max":"rev_bert_max","size":"rev_bert_cnt"
            })
        )
        train = train.merge(tr_agg, on="id", how="left")
        test  = test.merge(te_agg, on="id",  how="left")

        def mean_embed_by_id(ids, X):
            codes, uniques = pd.factorize(ids)
            K = len(uniques); D = X.shape[1]
            sum_mat = np.zeros((K, D), dtype=np.float32)
            cnt = np.zeros(K, dtype=np.int32)
            for i, c in enumerate(codes):
                if c >= 0:
                    sum_mat[c] += X[i]; cnt[c] += 1
            cnt[cnt == 0] = 1
            mean_mat = sum_mat / cnt[:, None]
            id2row = {u: i for i, u in enumerate(uniques)}
            return mean_mat, id2row

        tr_mean_emb, tr_map = mean_embed_by_id(tr_rev["id"].values, X_tr_rev)
        te_mean_emb, te_map = mean_embed_by_id(te_rev["id"].values, X_te_rev)

        D = tr_mean_emb.shape[1]
        train_mean = np.zeros((len(train), D), dtype=np.float32)
        test_mean  = np.zeros((len(test),  D), dtype=np.float32)
        for i, pid in enumerate(train["id"].values):
            row = tr_map.get(pid, None)
            if row is not None: train_mean[i] = tr_mean_emb[row]
        for i, pid in enumerate(test["id"].values):
            row = te_map.get(pid, None)
            if row is not None: test_mean[i] = te_mean_emb[row]

        all_mean = np.vstack([train_mean, test_mean])

        if BERT_SVD_COMPONENTS and all_mean.shape[1] > BERT_SVD_COMPONENTS:
            bert_svd = TruncatedSVD(n_components=BERT_SVD_COMPONENTS, random_state=RANDOM_STATE)
            all_svd = bert_svd.fit_transform(all_mean)
            tr_svd = all_svd[:len(train)]; te_svd = all_svd[len(train):]
            bert_svd_cols = [f"bert_svd_{i}" for i in range(BERT_SVD_COMPONENTS)]
            for i, col in enumerate(bert_svd_cols):
                train[col] = tr_svd[:, i]; test[col] = te_svd[:, i]
        else:
            bert_svd_cols = []

        raw_k = min(BERT_RAW_KEEP_DIMS, all_mean.shape[1])
        tr_raw = all_mean[:len(train), :raw_k]
        te_raw = all_mean[len(train):, :raw_k]
        bert_raw_cols = [f"bert_raw_{i}" for i in range(raw_k)]
        for i, col in enumerate(bert_raw_cols):
            train[col] = tr_raw[:, i]; test[col] = te_raw[:, i]

        del X_tr_rev, X_te_rev, tr_mean_emb, te_mean_emb, train_mean, test_mean, all_mean
        gc.collect()


In [ ]:
def add_samecat_knn_distance(train_df, test_df,
                             ks=(1, 3, 5),
                             lat_col="lat", lon_col="lon", cat_col="category"):
    all_df = pd.concat([
        train_df[[lat_col, lon_col, cat_col]].copy(),
        test_df[[lat_col, lon_col, cat_col]].copy()
    ], axis=0, ignore_index=True)

    cats = all_df[cat_col].astype(str).fillna("Unknown").values
    lat = pd.to_numeric(all_df[lat_col], errors="coerce").values
    lon = pd.to_numeric(all_df[lon_col], errors="coerce").values
    lat_rad = np.deg2rad(lat); lon_rad = np.deg2rad(lon)
    valid = np.isfinite(lat_rad) & np.isfinite(lon_rad)

    n_all = len(all_df)
    out_min  = np.full(n_all, np.nan, dtype=np.float32)
    out_k3   = np.full(n_all, np.nan, dtype=np.float32)
    out_k5   = np.full(n_all, np.nan, dtype=np.float32)

    uniq_cats = pd.unique(cats[valid])
    for cat in uniq_cats:
        mask_cat = (cats == cat) & valid
        idx_cat = np.where(mask_cat)[0]
        if idx_cat.size <= 1:
            continue

        X = np.c_[lat_rad[idx_cat], lon_rad[idx_cat]]
        tree = BallTree(X, metric="haversine")

        def knn_mean(k_need):
            kq = min(k_need + 1, X.shape[0])
            dist_rad, _ = tree.query(X, k=kq)
            dist_rad = dist_rad[:, 1:]
            if dist_rad.shape[1] == 0:
                return np.full(X.shape[0], np.nan, dtype=np.float32)
            return (dist_rad.mean(axis=1) * R_EARTH).astype(np.float32)

        k1q = min(2, X.shape[0])
        d1_rad, _ = tree.query(X, k=k1q)
        if k1q > 1:
            d1 = (d1_rad[:, 1] * R_EARTH).astype(np.float32)
            out_min[idx_cat] = d1

        out_k3[idx_cat] = knn_mean(3)
        out_k5[idx_cat] = knn_mean(5)

    n_tr = len(train_df)
    train_df["dist_samecat_min_m"]     = out_min[:n_tr]
    train_df["dist_samecat_mean_k3_m"] = out_k3[:n_tr]
    train_df["dist_samecat_mean_k5_m"] = out_k5[:n_tr]

    test_df["dist_samecat_min_m"]      = out_min[n_tr:]
    test_df["dist_samecat_mean_k3_m"]  = out_k3[n_tr:]
    test_df["dist_samecat_mean_k5_m"]  = out_k5[n_tr:]
    
add_samecat_knn_distance(train, test, ks=(1,3,5))

In [ ]:
def _make_grid_key(df, step_deg=0.01):
    lat = pd.to_numeric(df["lat"], errors="coerce").to_numpy()
    lon = pd.to_numeric(df["lon"], errors="coerce").to_numpy()
    cat = df["category"].astype("string").fillna("Unknown").to_numpy(dtype="U")

    ok = np.isfinite(lat) & np.isfinite(lon)

    glat = np.empty_like(lat); glat.fill(np.nan)
    glon = np.empty_like(lon); glon.fill(np.nan)
    glat[ok] = np.floor(lat[ok] / step_deg)
    glon[ok] = np.floor(lon[ok] / step_deg)

    key = np.full(len(df), None, dtype=object)
    if np.any(ok):
        lat_str = glat[ok].astype(np.int64).astype("U")
        lon_str = glon[ok].astype(np.int64).astype("U")
        cat_ok  = cat[ok].astype("U")

        s = np.char.add(np.char.add(np.char.add(lat_str, "_"), lon_str), "_")
        key[ok] = np.char.add(s, cat_ok)

    return key



def add_oof_grid_cat_mean(train_df, test_df, y_col="target",
                          steps=(0.01, 0.02),
                          prior=10.0, n_splits=5, seed=42):
    y = pd.to_numeric(train_df[y_col], errors="coerce").values.astype(float)
    bins = pd.qcut(y, q=min(10, max(2, len(y)//50)), duplicates="drop")
    y_bins = pd.factorize(bins)[0]
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    global_mean = np.nanmean(y)

    for step in steps:
        tag = str(step).replace(".", "p")
        k_tr_all = _make_grid_key(train_df, step_deg=step)
        k_te     = _make_grid_key(test_df,  step_deg=step)

        oof_mean = np.full(len(train_df), np.nan, dtype=np.float32)
        oof_cnt  = np.zeros(len(train_df), dtype=np.float32)

        for tr_idx, va_idx in skf.split(train_df, y_bins):
            k_tr = k_tr_all[tr_idx]; y_tr = y[tr_idx]
            g = pd.DataFrame({"k": k_tr, "y": y_tr}).dropna()
            gb = g.groupby("k")["y"].agg(["sum","count"])
            sm = gb["sum"].to_dict(); ct = gb["count"].to_dict()

            keys_va = k_tr_all[va_idx]
            m = np.array([(sm.get(k, 0.0) + prior*global_mean) / (ct.get(k, 0.0) + prior) if k is not None else global_mean
                          for k in keys_va], dtype=np.float32)
            c = np.array([ct.get(k, 0.0) if k is not None else 0.0 for k in keys_va], dtype=np.float32)

            oof_mean[va_idx] = m
            oof_cnt[va_idx]  = c

        train_df[f"oof_grid{tag}_cat_mean"] = oof_mean
        train_df[f"grid{tag}_cat_cnt"]      = oof_cnt

        # фичи для test — по full train
        gfull = pd.DataFrame({"k": k_tr_all, "y": y}).dropna()
        gbf = gfull.groupby("k")["y"].agg(["sum","count"])
        smf = gbf["sum"].to_dict(); ctf = gbf["count"].to_dict()
        te_mean = np.array([(smf.get(k, 0.0) + prior*global_mean) / (ctf.get(k, 0.0) + prior) if k is not None else global_mean
                            for k in k_te], dtype=np.float32)
        te_cnt  = np.array([ctf.get(k, 0.0) if k is not None else 0.0 for k in k_te], dtype=np.float32)

        test_df[f"oof_grid{tag}_cat_mean"] = te_mean
        test_df[f"grid{tag}_cat_cnt"]      = te_cnt

add_oof_grid_cat_mean(train, test, steps=(0.01, 0.02), prior=12.0, n_splits=N_FOLDS, seed=RANDOM_STATE)

In [ ]:
def add_metro_access_score(train_df, test_df, poi_df, radius_m=1500, eps=50.0, prefix="metro"):
    poi_lat = np.deg2rad(pd.to_numeric(poi_df["lat"], errors="coerce").values)
    poi_lon = np.deg2rad(pd.to_numeric(poi_df["lon"], errors="coerce").values)
    ok = np.isfinite(poi_lat) & np.isfinite(poi_lon)
    X_poi = np.c_[poi_lat[ok], poi_lon[ok]]
    if X_poi.shape[0] == 0:
        return
    tree = BallTree(X_poi, metric="haversine")

    def process(df):
        la = pd.to_numeric(df["lat"], errors="coerce").values
        lo = pd.to_numeric(df["lon"], errors="coerce").values
        v = np.isfinite(la) & np.isfinite(lo)
        X = np.c_[np.deg2rad(np.where(v, la, 0.0)), np.deg2rad(np.where(v, lo, 0.0))]
        ind, dist = tree.query_radius(X[v], r=radius_m/R_EARTH, return_distance=True, sort_results=True)
        invsum = np.zeros(np.sum(v), dtype=np.float32)
        inv2sum = np.zeros(np.sum(v), dtype=np.float32)
        for i, d in enumerate(dist):
            m = d * R_EARTH
            invsum[i] = np.sum(1.0 / (eps + m))
            inv2sum[i] = np.sum(1.0 / ((eps + m)**2))
        out1 = np.full(len(df), np.nan, dtype=np.float32); out1[v] = invsum
        out2 = np.full(len(df), np.nan, dtype=np.float32); out2[v] = inv2sum
        df[f"{prefix}_invsum_r{radius_m}"]  = out1
        df[f"{prefix}_inv2sum_r{radius_m}"] = out2

    process(train_df); process(test_df)

add_metro_access_score(train, test, metro_df, radius_m=1500, eps=50.0, prefix="metro")

In [ ]:
def add_center_rings(train_df, test_df, rings_km=(3,6,10,15)):
    for df in (train_df, test_df):
        d = pd.to_numeric(df["dist_to_center_m"], errors="coerce")
        df["center_logdist"]   = np.log1p(d)
        df["center_invdist_km"] = 1.0 / (1e-3 + d/1000.0)
        bins = [0] + [r*1000 for r in rings_km] + [np.inf]
        df["center_ring_id"] = pd.cut(d, bins=bins, labels=False, right=True).astype("float32")

        if "traffic_300m" in df.columns:
            df["center_invdist_x_traffic"] = df["center_invdist_km"] * pd.to_numeric(df["traffic_300m"], errors="coerce")
        if "samecat_cnt_r300" in df.columns:
            df["center_invdist_x_comp300"] = df["center_invdist_km"] * pd.to_numeric(df["samecat_cnt_r300"], errors="coerce")

add_center_rings(train, test)

In [ ]:
def add_text_geo_interactions(df):
    for tcol in [c for c in df.columns if c in ("tfidf_pls16_oof","tfidf_ridge_oof","tokte_oof","text_cb_oof")]:
        if "center_invdist_km" in df.columns:
            df[f"{tcol}_x_centerinv"] = df[tcol] * df["center_invdist_km"]
        if "metro_invsum_r1500" in df.columns:
            df[f"{tcol}_x_metroacc"] = df[tcol] * df["metro_invsum_r1500"]
        if "samecat_cnt_r300" in df.columns:
            df[f"{tcol}_x_comp300"] = df[tcol] * df["samecat_cnt_r300"]

add_text_geo_interactions(train); add_text_geo_interactions(test)

In [ ]:
def add_oof_linear_head(train_df, test_df, feature_cols, y, name="tfidf_ridge",
                        alpha=3.0, n_splits=5, seed=42):
    X_tr = train_df[feature_cols].astype(np.float32).fillna(0.0).to_numpy()
    X_te = test_df[feature_cols].astype(np.float32).fillna(0.0).to_numpy()
    y_all = np.asarray(pd.to_numeric(y, errors="coerce"), dtype=np.float32)

    bins = pd.qcut(y_all, q=min(10, max(2, len(y_all)//50)), duplicates="drop")
    y_bins = pd.factorize(bins)[0]
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    oof = np.zeros(len(train_df), dtype=np.float32)
    te  = np.zeros(len(test_df),  dtype=np.float32)

    for tr_idx, va_idx in skf.split(X_tr, y_bins):
        mdl = Ridge(alpha=alpha, solver="auto", random_state=seed)
        mdl.fit(X_tr[tr_idx], y_all[tr_idx])
        oof[va_idx] = mdl.predict(X_tr[va_idx]).astype(np.float32)
        te += (mdl.predict(X_te).astype(np.float32) / n_splits)

    train_df[f"{name}_oof"] = oof
    test_df[f"{name}_pred"] = te

# вызов
svd_cols = [c for c in train.columns if c.startswith("text_svd_")]
add_oof_linear_head(train, test, svd_cols, y=train["target"].values, name="tfidf_ridge", alpha=3.0,n_splits=N_FOLDS, seed=RANDOM_STATE)

In [ ]:
def add_oof_pls_head(train_df, test_df, feature_cols, y, n_comp=16,
                     name="pls", n_splits=5, seed=42):
    X_tr = train_df[feature_cols].astype(np.float32).fillna(0.0).to_numpy()
    X_te = test_df[feature_cols].astype(np.float32).fillna(0.0).to_numpy()
    y_all = np.asarray(pd.to_numeric(y, errors="coerce"), dtype=np.float32)

    bins = pd.qcut(y_all, q=min(10, max(2, len(y_all)//50)), duplicates="drop")
    y_bins = pd.factorize(bins)[0]
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    oof = np.zeros(len(train_df), dtype=np.float32)
    te  = np.zeros(len(test_df),  dtype=np.float32)

    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_tr, y_bins), 1):
        xs = StandardScaler(with_mean=True, with_std=True)
        ys = StandardScaler(with_mean=True, with_std=True)

        Xtr_f = xs.fit_transform(X_tr[tr_idx])
        Xva_f = xs.transform(X_tr[va_idx])
        Xte_f = xs.transform(X_te)

        ytr_f = ys.fit_transform(y_all[tr_idx].reshape(-1, 1)).ravel()

        n_comp_eff = int(min(n_comp, Xtr_f.shape[1], len(tr_idx) - 1))
        if n_comp_eff < 1:
            oof[va_idx] = ys.inverse_transform(
                np.full((len(va_idx), 1), ytr_f.mean(), dtype=np.float32)
            )[:, 0]
            te += ys.inverse_transform(
                np.full((len(X_te), 1), ytr_f.mean(), dtype=np.float32)
            )[:, 0] / n_splits
            print(f"[PLS-{name}] fold {fold}: n_comp_eff<1, used mean fallback")
            continue

        pls = PLSRegression(n_components=n_comp_eff, scale=False)
        pls.fit(Xtr_f, ytr_f)

        oof_pred = ys.inverse_transform(pls.predict(Xva_f))[:, 0]
        te_pred  = ys.inverse_transform(pls.predict(Xte_f))[:, 0]

        oof[va_idx] = oof_pred.astype(np.float32)
        te += (te_pred.astype(np.float32) / n_splits)

        print(f"[PLS-{name}] fold {fold}: n_comp={n_comp_eff}, "
              f"train={len(tr_idx)}, val={len(va_idx)}")

    train_df[f"{name}_oof"] = oof
    test_df[f"{name}_pred"] = te



In [ ]:
svd_cols = [c for c in train.columns if c.startswith("text_svd_")]


for k in (10, 16, 22):
    add_oof_pls_head(train, test, svd_cols, y=train["target"].astype(float).values,
                     n_comp=k, name=f"tfidf_pls{k}", n_splits=N_FOLDS, seed=RANDOM_STATE)

    
bert_pls_cols = [c for c in train.columns if c.startswith("bert_svd_") or c.startswith("bert_raw_")]
if len(bert_pls_cols) >= 8:
    add_oof_pls_head(
        train, test,
        feature_cols=bert_pls_cols,
        y=train["target"].astype(float).values,
        n_comp=8,
        name="bert_pls8",
        n_splits=N_FOLDS, seed=RANDOM_STATE
    )

In [ ]:
def add_oof_pls_per_category(train_df, test_df, feature_cols, y,
                             cat_col="category", min_size=250,
                             n_comp=12, name="tfidf_pls_cat",
                             n_splits=5, seed=42):
    add_oof_pls_head(train_df, test_df, feature_cols, y,
                     n_comp=n_comp, name=f"{name}_global",
                     n_splits=n_splits, seed=seed)

    key = train_df[cat_col].astype(str).fillna("Unknown")
    sizes = key.value_counts()
    big = set(sizes[sizes >= min_size].index)

    oof = train_df[f"{name}_global_oof"].copy().to_numpy()
    pred = test_df[f"{name}_global_pred"].copy().to_numpy()

    for cat in big:
        m_tr = key == cat
        m_te = test_df[cat_col].astype(str).fillna("Unknown") == cat
        if m_tr.sum() < n_splits + 5:
            continue
        add_oof_pls_head(train_df[m_tr], test_df[m_te], feature_cols, y[m_tr],
                         n_comp=n_comp, name=f"{name}_{cat}", n_splits=n_splits, seed=seed)
        oof[m_tr] = train_df.loc[m_tr, f"{name}_{cat}_oof"].to_numpy()
        pred[m_te] = test_df.loc[m_te,  f"{name}_{cat}_pred"].to_numpy()

    train_df[f"{name}_oof"] = oof.astype(np.float32)
    test_df[f"{name}_pred"] = pred.astype(np.float32)

In [ ]:
def add_oof_token_score(train_df, test_df, text_col="reviews_text",
                        y_col="target", name="tokte",
                        min_df=20, max_df=0.9, ngram_range=(1,2),
                        prior=12.0, n_splits=5, seed=42):
    vect = CountVectorizer(min_df=min_df, max_df=max_df, ngram_range=ngram_range,
                           token_pattern=r"(?u)\b[а-яa-z0-9][а-яa-z0-9\-]{2,}\b",
                           lowercase=True, binary=False)
    X_all = vect.fit_transform(pd.concat([train_df[text_col], test_df[text_col]]).astype(str).values)
    X_tr = X_all[:len(train_df)]
    X_te = X_all[len(train_df):]
    y = pd.to_numeric(train_df[y_col], errors="coerce").values.astype(np.float32)
    global_mean = float(np.nanmean(y))

    bins = pd.qcut(y, q=min(10, max(2, len(y)//50)), duplicates="drop")
    y_bins = pd.factorize(bins)[0]
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    oof_score = np.zeros(len(train_df), dtype=np.float32); te_score = np.zeros(len(test_df), dtype=np.float32)

    for tr_idx, va_idx in skf.split(X_tr, y_bins):
        Xtr = X_tr[tr_idx]; ytr = y[tr_idx]
        term_sum = np.asarray(Xtr.T @ ytr).ravel().astype(np.float64)
        term_cnt = np.asarray(Xtr.T.sum(axis=1)).ravel().astype(np.float64)
        term_mean = (term_sum + prior * global_mean) / (term_cnt + prior)

        def doc_score(X, w):
            num = np.asarray(X @ w).ravel()
            den = np.asarray(X.sum(axis=1)).ravel() + 1e-6
            return (num / den).astype(np.float32)

        oof_score[va_idx] = doc_score(X_tr[va_idx], term_mean)
        te_score += doc_score(X_te, term_mean) / n_splits

    train_df[f"{name}_oof"] = oof_score
    test_df[f"{name}_pred"] = te_score

In [ ]:
def _normalize_chain_name(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"[\"«».,!?:;()\-–—\[\]/\\]+", " ", s)
    s = re.sub(r"\b(ооо|ип|зао|оао|пао|ao|ooo|zao|oao|ooo\s+|ооо\s+)\b\.?", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

for df in (train, test):
    df["name_norm"] = df.get("name", "").astype(str).map(_normalize_chain_name).fillna("unknown")

In [ ]:
name_counts = pd.concat([train["name_norm"], test["name_norm"]]).value_counts()
for df in (train, test):
    df["chain_count_all"] = df["name_norm"].map(name_counts).astype("float32")
    df["chain_is_big_3+"] = (df["chain_count_all"] >= 3).astype("int8")
    df["chain_is_big_5+"] = (df["chain_count_all"] >= 5).astype("int8")


In [ ]:
def add_oof_mean_by_key(train_df, test_df, key_cols, y_col="target",
                        prior=15.0, name="te", n_splits=5, seed=42):
    key = train_df[key_cols].astype(str).fillna("Unknown").agg("||".join, axis=1).values
    key_te = test_df[key_cols].astype(str).fillna("Unknown").agg("||".join, axis=1).values
    y = pd.to_numeric(train_df[y_col], errors="coerce").values.astype(np.float32)
    gmean = float(np.nanmean(y))
    bins = pd.qcut(y, q=min(10, max(2, len(y)//50)), duplicates="drop")
    y_bins = pd.factorize(bins)[0]
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    oof = np.zeros(len(train_df), dtype=np.float32)
    te  = np.zeros(len(test_df),  dtype=np.float32)

    for tr_idx, va_idx in skf.split(train_df, y_bins):
        k_tr = key[tr_idx]; y_tr = y[tr_idx]
        df = pd.DataFrame({"k": k_tr, "y": y_tr})
        gb = df.groupby("k")["y"].agg(["sum","count"])
        s = gb["sum"].to_dict(); c = gb["count"].to_dict()
        def map_keys(keys):
            return np.array([(s.get(k,0.0) + prior*gmean) / (c.get(k,0.0) + prior) for k in keys], dtype=np.float32)
        oof[va_idx] = map_keys(key[va_idx])
        te += map_keys(key_te) / n_splits

    train_df[f"{name}_oof"] = oof
    test_df[f"{name}_pred"] = te

add_oof_mean_by_key(train, test, ["name_norm"],name="chain_te", prior=20.0, n_splits=N_FOLDS, seed=RANDOM_STATE)
if "category" in train.columns:
    add_oof_mean_by_key(train, test, ["name_norm","category"], name="chaincat_te", prior=20.0, n_splits=N_FOLDS, seed=RANDOM_STATE)

In [ ]:
def add_same_name_counts(train_df, test_df, name_col="name_norm", radii=(300, 1000)):
    all_df = pd.concat([train_df[["lat","lon",name_col]],
                        test_df[["lat","lon",name_col]]], ignore_index=True)
    lat = pd.to_numeric(all_df["lat"], errors="coerce").values
    lon = pd.to_numeric(all_df["lon"], errors="coerce").values
    ok = np.isfinite(lat) & np.isfinite(lon)
    lat_r = np.deg2rad(np.where(ok, lat, 0.0)); lon_r = np.deg2rad(np.where(ok, lon, 0.0))
    names = all_df[name_col].astype(str).fillna("unknown").values
    out = {r: np.full(len(all_df), np.nan, dtype=np.float32) for r in radii}

    for nm in pd.unique(names[ok]):
        idx = np.where((names==nm) & ok)[0]
        if idx.size <= 1: 
            for r in radii: out[r][idx] = 0
            continue
        X = np.c_[lat_r[idx], lon_r[idx]]
        tree = BallTree(X, metric="haversine")
        for r in radii:
            inds = tree.query_radius(X, r=r/R_EARTH, return_distance=False)
            cnt = np.fromiter((max(0, len(ii)-1) for ii in inds), dtype=np.int32)
            out[r][idx] = cnt.astype(np.float32)

    ntr = len(train_df)
    for r in radii:
        train_df[f"same_name_cnt_r{r}"] = out[r][:ntr]
        test_df[f"same_name_cnt_r{r}"]  = out[r][ntr:]

add_same_name_counts(train, test, name_col="name_norm", radii=(300, 1000))

In [ ]:
def add_center_angle_features(train_df, test_df, n_sectors=16):
    for df in (train_df, test_df):
        lat = pd.to_numeric(df["lat"], errors="coerce").values
        lon = pd.to_numeric(df["lon"], errors="coerce").values
        ang = np.arctan2(lat - MOSCOW_CENTER_LAT, lon - MOSCOW_CENTER_LON)
        ang[~np.isfinite(ang)] = np.nan
        df["center_angle_sin"] = np.sin(ang).astype("float32")
        df["center_angle_cos"] = np.cos(ang).astype("float32")
        sec = np.floor(((ang + np.pi) / (2*np.pi)) * n_sectors)
        df[f"center_sector_{n_sectors}"] = sec.astype("float32")

add_center_angle_features(train, test, n_sectors=16)

In [ ]:
def add_kmeans_cluster_te(train_df, test_df, n_clusters=60, seed=42):
    xy_tr = train_df[["lat","lon"]].apply(pd.to_numeric, errors="coerce")
    xy_te = test_df[["lat","lon"]].apply(pd.to_numeric, errors="coerce")
    scaler = StandardScaler()
    xy_all = pd.concat([xy_tr, xy_te], axis=0).to_numpy()
    mask = np.isfinite(xy_all).all(axis=1)
    xy_scaled = np.zeros_like(xy_all, dtype=np.float32)
    xy_scaled[mask] = scaler.fit_transform(xy_all[mask])

    km = KMeans(n_clusters=n_clusters, random_state=seed, n_init=10)
    lab = np.full(len(xy_all), -1, dtype=np.int32)
    lab[mask] = km.fit_predict(xy_scaled[mask])
    lab_tr = lab[:len(train_df)]; lab_te = lab[len(train_df):]

    train_df[f"kmeans{n_clusters}"] = lab_tr
    test_df[f"kmeans{n_clusters}"]  = lab_te

    add_oof_mean_by_key(train_df, test_df, [f"kmeans{n_clusters}"], name=f"kmeans{n_clusters}_te",
                        prior=12.0, n_splits=N_FOLDS, seed=RANDOM_STATE)

add_kmeans_cluster_te(train, test, n_clusters=60, seed=RANDOM_STATE)

In [ ]:
def add_local_market_structure(train_df, test_df, radii=(200, 300, 500)):
    all_df = pd.concat([train_df[["lat","lon","category"]],
                        test_df[["lat","lon","category"]]], ignore_index=True)
    lat = pd.to_numeric(all_df["lat"], errors="coerce").values
    lon = pd.to_numeric(all_df["lon"], errors="coerce").values
    ok = np.isfinite(lat) & np.isfinite(lon)
    X = np.c_[np.deg2rad(np.where(ok, lat, 0.0)), np.deg2rad(np.where(ok, lon, 0.0))]
    cats = all_df["category"].astype(str).fillna("Unknown").values
    tree = BallTree(X[ok], metric="haversine")

    res = {r: dict(all=np.full(len(all_df), np.nan, dtype=np.float32),
                   uniq=np.full(len(all_df), np.nan, dtype=np.float32)) for r in radii}

    for r in radii:
        ind = tree.query_radius(X[ok], r=r/R_EARTH, return_distance=False)
        cnt_all = np.zeros(np.sum(ok), dtype=np.int32)
        cnt_uniq= np.zeros(np.sum(ok), dtype=np.int32)
        for i, nb in enumerate(ind):
            gidx = np.where(ok)[0][i]
            nb = np.where(ok)[0][nb]
            nb = nb[nb != gidx]
            cnt_all[i] = len(nb)
            if len(nb)>0:
                cnt_uniq[i] = len(np.unique(cats[nb]))
        res[r]["all"][ok]  = cnt_all
        res[r]["uniq"][ok] = cnt_uniq

    ntr = len(train_df)
    for r in radii:
        train_df[f"allcat_cnt_r{r}"]  = res[r]["all"][:ntr]
        test_df[f"allcat_cnt_r{r}"]   = res[r]["all"][ntr:]
        train_df[f"uniqcat_cnt_r{r}"] = res[r]["uniq"][:ntr]
        test_df[f"uniqcat_cnt_r{r}"]  = res[r]["uniq"][ntr:]
        for df in (train_df, test_df):
            if f"samecat_cnt_r{r}" in df.columns:
                df[f"same_over_all_r{r}"] = _safe_div(df[f"samecat_cnt_r{r}"], df[f"allcat_cnt_r{r}"])
add_local_market_structure(train, test, radii=(200, 300))

In [ ]:
if "metro_dist_min_m" in train.columns and "category" in train.columns:
    med = train.groupby("category")["metro_dist_min_m"].median()
    for df in (train, test):
        df["cat_metro_med"] = df["category"].map(med).astype("float32")
        df["metro_minus_cat_med"] = (pd.to_numeric(df["metro_dist_min_m"], errors="coerce") - df["cat_metro_med"]).astype("float32")
        df["metro_ratio_cat_med"] = _safe_div(df["metro_dist_min_m"], df["cat_metro_med"]).astype("float32")

In [ ]:
def oof_local_target_stats_any(train_df, test_df, radii_m=(200,300), n_splits=5, seed=42):
    y = train_df["target"].astype(float).values
    lat_tr = pd.to_numeric(train_df["lat"], errors="coerce").values
    lon_tr = pd.to_numeric(train_df["lon"], errors="coerce").values
    lat_te = pd.to_numeric(test_df["lat"],  errors="coerce").values
    lon_te = pd.to_numeric(test_df["lon"],  errors="coerce").values

    bins = pd.qcut(y, q=min(10, max(2, len(y)//50)), duplicates="drop")
    y_bins = pd.factorize(bins)[0]
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    valid_tr = np.isfinite(lat_tr) & np.isfinite(lon_tr)
    valid_te = np.isfinite(lat_te) & np.isfinite(lon_te)
    X_tr_all = np.c_[np.deg2rad(np.where(valid_tr, lat_tr, 0.0)),
                     np.deg2rad(np.where(valid_tr, lon_tr, 0.0))]
    X_te_all = np.c_[np.deg2rad(np.where(valid_te, lat_te, 0.0)),
                     np.deg2rad(np.where(valid_te, lon_te, 0.0))]

    for r_m in radii_m:
        r = r_m / R_EARTH
        oof_mean = np.full(len(train_df), np.nan, dtype=np.float32)
        oof_med  = np.full(len(train_df), np.nan, dtype=np.float32)
        oof_cnt  = np.zeros(len(train_df), dtype=np.float32)

        for tr_idx, va_idx in skf.split(train_df, y_bins):
            tr_idx_v = tr_idx[valid_tr[tr_idx]]
            va_idx_v = va_idx[valid_tr[va_idx]]
            if tr_idx_v.size == 0 or va_idx_v.size == 0: continue
            Xtr, ytr = X_tr_all[tr_idx_v], y[tr_idx_v]
            tree = BallTree(Xtr, metric="haversine")
            neigh = tree.query_radius(X_tr_all[va_idx_v], r=r, return_distance=False)
            for j, nb in enumerate(neigh):
                if nb.size == 0: continue
                vals = ytr[nb]
                oof_mean[va_idx_v[j]] = float(np.mean(vals))
                oof_med[va_idx_v[j]]  = float(np.median(vals))
                oof_cnt[va_idx_v[j]]  = float(len(vals))

        train_df[f"loc_any_mean_r{r_m}"]   = oof_mean
        train_df[f"loc_any_median_r{r_m}"] = oof_med
        train_df[f"loc_any_count_r{r_m}"]  = oof_cnt

        # test на полном train
        tr_all_v = np.where(valid_tr)[0]
        if tr_all_v.size and np.any(valid_te):
            Xtr, ytr = X_tr_all[tr_all_v], y[tr_all_v]
            tree = BallTree(Xtr, metric="haversine")
            te_v = np.where(valid_te)[0]
            neigh_te = tree.query_radius(X_te_all[te_v], r=r, return_distance=False)
            te_mean = np.full(len(test_df), np.nan, dtype=np.float32)
            te_med  = np.full(len(test_df), np.nan, dtype=np.float32)
            te_cnt  = np.zeros(len(test_df), dtype=np.float32)
            for j, nb in enumerate(neigh_te):
                if nb.size == 0: continue
                vals = ytr[nb]
                idx = te_v[j]
                te_mean[idx] = float(np.mean(vals))
                te_med[idx]  = float(np.median(vals))
                te_cnt[idx]  = float(len(vals))
            test_df[f"loc_any_mean_r{r_m}"]   = te_mean
            test_df[f"loc_any_median_r{r_m}"] = te_med
            test_df[f"loc_any_count_r{r_m}"]  = te_cnt

oof_local_target_stats_any(train, test, radii_m=(200,300), n_splits=N_FOLDS, seed=RANDOM_STATE)

In [ ]:
def add_same_name_distance(train_df, test_df, name_col="name_norm", ks=(1,3)):
    all_df = pd.concat([train_df[["lat","lon",name_col]],
                        test_df[["lat","lon",name_col]]], ignore_index=True)
    lat = pd.to_numeric(all_df["lat"], errors="coerce").values
    lon = pd.to_numeric(all_df["lon"], errors="coerce").values
    ok = np.isfinite(lat) & np.isfinite(lon)
    lat_r = np.deg2rad(np.where(ok, lat, 0.0)); lon_r = np.deg2rad(np.where(ok, lon, 0.0))
    names = all_df[name_col].astype(str).fillna("unknown").values

    out_min = np.full(len(all_df), np.nan, dtype=np.float32)
    out_k3  = np.full(len(all_df), np.nan, dtype=np.float32)

    for nm in pd.unique(names[ok]):
        idx = np.where((names==nm) & ok)[0]
        if idx.size <= 1: 
            out_min[idx] = np.nan; out_k3[idx] = np.nan
            continue
        X = np.c_[lat_r[idx], lon_r[idx]]
        tree = BallTree(X, metric="haversine")
        kq = min(2, X.shape[0])
        d2, _ = tree.query(X, k=kq)
        if kq > 1:
            out_min[idx] = (d2[:,1] * R_EARTH).astype(np.float32)
        kq = min(4, X.shape[0])
        d4, _ = tree.query(X, k=kq)
        if kq > 1:
            dnn = d4[:,1:] * R_EARTH
            out_k3[idx] = dnn.mean(axis=1).astype(np.float32)

    ntr = len(train_df)
    train_df["same_name_dist_min_m"]   = out_min[:ntr]
    test_df["same_name_dist_min_m"]    = out_min[ntr:]
    train_df["same_name_dist_mean3_m"] = out_k3[:ntr]
    test_df["same_name_dist_mean3_m"]  = out_k3[ntr:]

add_same_name_distance(train, test, name_col="name_norm", ks=(1,3))

In [ ]:
def add_category_diversity(train_df, test_df, radii=(200,300)):
    all_df = pd.concat([train_df[["lat","lon","category"]],
                        test_df[["lat","lon","category"]]], ignore_index=True)
    lat = pd.to_numeric(all_df["lat"], errors="coerce").values
    lon = pd.to_numeric(all_df["lon"], errors="coerce").values
    cats = all_df["category"].astype(str).fillna("Unknown").values
    ok = np.isfinite(lat) & np.isfinite(lon)
    X = np.c_[np.deg2rad(np.where(ok, lat, 0.0)), np.deg2rad(np.where(ok, lon, 0.0))]
    tree = BallTree(X[ok], metric="haversine")

    for r in radii:
        ent = np.full(len(all_df), np.nan, dtype=np.float32)
        hhi = np.full(len(all_df), np.nan, dtype=np.float32)
        ind = tree.query_radius(X[ok], r=r/R_EARTH, return_distance=False)
        ok_idx = np.where(ok)[0]
        for i, nb_local in enumerate(ind):
            gidx = ok_idx[i]
            nb = ok_idx[nb_local]
            nb = nb[nb != gidx]
            if nb.size == 0: 
                ent[i] = np.nan; hhi[i] = np.nan; continue
            vals, cnts = np.unique(cats[nb], return_counts=True)
            p = cnts / cnts.sum()
            ent[i] = float(-(p * np.log(p + 1e-12)).sum())
            hhi[i] = float((p**2).sum())
        ntr = len(train_df)
        train_df[f"cat_entropy_r{r}"] = ent[:ntr]
        test_df[f"cat_entropy_r{r}"]  = ent[ntr:]
        train_df[f"cat_hhi_r{r}"]     = hhi[:ntr]
        test_df[f"cat_hhi_r{r}"]      = hhi[ntr:]

add_category_diversity(train, test, radii=(200,300))

In [ ]:
if "dist_to_center_m" in train.columns and "category" in train.columns:
    med_c = train.groupby("category")["dist_to_center_m"].median()
    for df in (train, test):
        df["cat_center_med"] = df["category"].map(med_c).astype("float32")
        d = pd.to_numeric(df["dist_to_center_m"], errors="coerce")
        cm = pd.to_numeric(df["cat_center_med"], errors="coerce")
        df["center_minus_cat_med"] = (d - cm).astype("float32")
        df["center_ratio_cat_med"] = _safe_div(d, cm).astype("float32")


In [ ]:
for col in [c for c in train.columns if c.endswith("_oof")]:
    pred = col.replace("_oof","_pred")
    if pred in test.columns:
        test[col] = test[pred]


In [ ]:
drop_cols = {"target","address","coordinates","reviews_text"}
base_cols = [c for c in train.columns if c not in drop_cols and c != "id"]

cat_cols = []
if "category" in train.columns: cat_cols.append("category")
if "name" in train.columns:     cat_cols.append("name")
cat_cols = [c for c in cat_cols if c in base_cols]

num_cols = [c for c in base_cols
            if c not in cat_cols
            and not c.startswith("kw_")
            and not c.startswith("rev_bert_")
            and not c.startswith("bert_svd_")
            and not c.startswith("bert_mean_")
            and not c.startswith("bert_raw_")]

for c in cat_cols:
    train[c] = train[c].astype(str).fillna("Unknown")
    test[c]  = test[c].astype(str).fillna("Unknown")
for c in num_cols:
    train[c] = pd.to_numeric(train[c], errors="coerce")
    test[c]  = pd.to_numeric(test[c], errors="coerce")

_ = light_log1p_skewed(train, num_cols, skew_thr=1.0)
_ = light_log1p_skewed(test,  num_cols, skew_thr=1.0)

kw_cols = [c for c in train.columns if c.startswith("kw_")]
bert_cols = [c for c in train.columns if c.startswith("rev_bert_") or c.startswith("bert_svd_") or c.startswith("bert_raw_")]

OOF_HEADS = [
    "tfidf_pls16",
    "bert_pls8",
    "tfidf_ridge",
    "tfidf_pls_cat",
    "tokte",
    "text_cb",
]
OOF_HEADS += ["chain_te", "chaincat_te"]
OOF_HEADS += [ "kmeans60_te" ]


if "category" in train.columns:
    add_oof_mean_by_key(train, test, ["category"], name="cat_te",
                        prior=10.0, n_splits=N_FOLDS, seed=RANDOM_STATE)
    OOF_HEADS += ["cat_te"]

if "center_ring_id" in train.columns and "category" in train.columns:
    add_oof_mean_by_key(train, test, ["category","center_ring_id"], name="cat_ring_te",
                        prior=12.0, n_splits=N_FOLDS, seed=RANDOM_STATE)
    OOF_HEADS += ["cat_ring_te"]


extra_oof_cols = [f"{h}_oof" for h in OOF_HEADS if f"{h}_oof" in train.columns]

all_cols = cat_cols + num_cols + kw_cols + bert_cols + extra_oof_cols

all_cols = pd.Index(all_cols).drop_duplicates().tolist()

te_cols = [c.replace("_oof", "_pred") for c in all_cols]

for c in te_cols:
    if c not in test.columns:
        test[c] = np.nan

X = train[all_cols].copy()
X = X.loc[:, ~X.columns.duplicated()]

X_test = test[te_cols].copy()
X_test.columns = X.columns


bins = pd.qcut(train["target"], q=min(10, train.shape[0]//50), duplicates="drop")
y_bins = pd.factorize(bins)[0]
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

oof = np.zeros(len(train), dtype=np.float32)
test_pred = np.zeros(len(test), dtype=np.float32)
task_type = "GPU" if USE_GPU else "CPU"

In [ ]:
print(f"Начинаем {N_FOLDS}-fold обучение CatBoost (task_type={task_type})...")
for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y_bins), 1):
    X_tr, y_tr = X.iloc[tr_idx], y[tr_idx]
    X_va, y_va = X.iloc[va_idx], y[va_idx]

    train_pool = Pool(X_tr, y_tr, cat_features=cat_cols)
    valid_pool = Pool(X_va, y_va, cat_features=cat_cols)
    test_pool  = Pool(X_test,      cat_features=cat_cols)

    model = CatBoostRegressor(
        loss_function="MAE", eval_metric="MAE",
        iterations=20000, depth=8, learning_rate=0.03064123,
        l2_leaf_reg=6.0, random_strength=1.5, bagging_temperature=0.5,
        od_type="Iter", od_wait=500, use_best_model=True,
        task_type=task_type, random_seed=RANDOM_STATE
    )
    model.fit(train_pool, eval_set=valid_pool, verbose=200)

    pred_va = model.predict(valid_pool).astype(np.float32)
    oof[va_idx] = pred_va
    fold_mae = mean_absolute_error(y_va, pred_va)
    print(f"[Fold {fold}] MAE: {fold_mae:.5f}, best_iter={model.get_best_iteration()}")

    test_pred += model.predict(test_pool).astype(np.float32) / N_FOLDS

oof_mae = mean_absolute_error(y, oof)
print(f"OOF MAE = {oof_mae:.6f}")

test_pred = np.clip(test_pred, 1.0, 5.0)
sub = pd.DataFrame({"id": test["id"], "target": test_pred})
sub.to_csv(SUBMISSION_PATH, index=False)
print(f"Сохранено: {SUBMISSION_PATH}")